In [2]:
#DATASET SINTÉTICO 3

import numpy as np
import pandas as pd


np.random.seed(42)


In [ ]:

#Numero de muestras

n_samples = 1000

# Variables exógenas  (no dependen de ninguna otra)

Z = np.random.normal(0,1,n_samples)  # confusor

X1 = 2*Z + np.random.normal(0,1,n_samples)
X2 = 0.67*Z + np.random.uniform(-2, 2, n_samples)
X3 = np.random.choice([0, 1], size= n_samples, p=[0.6, 0.4])

#Variables derivadas no lineales
Y1 = np.sin(np.pi * X1) + np.random.normal(0, 0.2, n_samples)


# Variables categóricas derivadas

def gen_Y2(X3, Y1):
    if X3 == 0 and Y1 < 0.5:
        return np.random.choice([0, 1], p=[0.7, 0.3])
    elif X3 == 1 and  X3 >= 0.5:
        return np.random.choice([0, 1], p=[0.2, 0.8])
    else:
        return np.random.choice([0, 1], p=[0.4, 0.6])
Y2 = np.array([gen_Y2(x3, y1) for x3, y1 in zip(X3, Y1)])

def gen_Y3(Y1, Y2): # Label
    if Y1 < 0.7 and Y2 == 0:
        return np.random.choice([0, 1], p=[0.5, 0.5])
    elif Y1 >=0.7 and  Y2 == 1:
        return np.random.choice([0, 1], p=[0.65, 0.35])
    else:
        return np.random.choice([0, 1], p=[0.1, 0.9])
Y3 = np.array([gen_Y3(y1, y2) for y1, y2 in zip(Y1, Y2)])

# Variables independientes o con ruido puro
Noise1 = np.random.normal(0, 1, n_samples)
Noise2 = np.random.normal(10, 1, n_samples)


df_three = pd.DataFrame({
    "Z": Z, "X1": X1, "X2": X2, "X3": X3,
    "Y1": Y1, "Y2": Y2, "Y3": Y3, "Noise1": Noise1, 
    "Noise2": Noise2
})

df_three.head()

In [3]:
# Normalizamos las variables continuas

from sklearn.preprocessing import StandardScaler, OneHotEncoder

scaler = StandardScaler()
numerical_cols = ["Z","X1", "X2", "Y1", "Noise1", "Noise2" ]
df_three[numerical_cols] = scaler.fit_transform(df_three[numerical_cols])

df_three.head()

,Z,X1,X2,X3,Y1,Y2,Y3,Noise1,Noise2
0,0.487759,1.056820,-0.028672,0,1.541848,1,0,-0.117110,1.038345
1,-0.161022,0.249294,-1.340393,0,1.349954,0,1,0.460532,0.029623
2,0.642015,0.576484,-0.125405,1,-0.598750,1,1,-0.811824,-0.473659
3,1.536382,1.059754,-0.392767,0,0.866472,1,1,0.166196,-0.245314
4,-0.258995,0.055735,0.788366,0,1.213642,1,1,-0.328335,-0.169848


In [7]:
from causallearn.search.ConstraintBased.FCI import fci

node_names = list(df_three.columns)  # nombres de las variables
data_three = df_three.to_numpy()

graph, edges = fci(
        data_three,
        independence_test_method='kci',
        alpha=0.05,
        depth=2,
        verbose=False,
        node_names=node_names,
        kernelX="Gaussian",
        kernelY="Gaussian",
        kernelZ="Gaussian",
        print_edges_to_trace=[('X1','X2')]
)

Depth=0, working on node 8: 100%|██████████| 9/9 [00:02<00:00,  4.18it/s]


Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA
Arista X1-X2: ELIMINADA


In [3]:
np.random.seed(42)
N = 300
L = np.random.randn(N)
X1 = 2 * L + np.random.randn(N) * 0.2
X2 = -3 * L + np.random.randn(N) * 0.2
X3 = np.random.randn(N) * 5  # Ruido sin relación

df_latent3 = pd.DataFrame({'X1': X1, 'X2': X2, 'X3': X3})

df_latent3.head()



,X1,X2,X3
0,0.827629,-1.338745,1.843367
1,-0.388565,0.230360,-1.966694
2,1.444836,-1.769144,0.143724
3,3.168134,-4.297962,6.392259
4,-0.472487,0.785147,0.955495


In [6]:
from causallearn.search.ConstraintBased.FCI import fci

node_names = list(df_latent3.columns)  # nombres de las variables
data_latent = df_latent3.to_numpy()

graph, edges = fci(
        data_latent,
        independence_test_method='kci',
        alpha=0.05,
        depth=2,
        verbose=False,
        node_names=node_names,
        kernelX="Gaussian",
        kernelY="Gaussian",
        kernelZ="Gaussian",
        print_edges_to_trace=[('X2','X3')]
)

Depth=0, working on node 2: 100%|██████████| 3/3 [00:00<00:00, 302.47it/s]


Arista X2-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X2-X3: ELIMINADA
Arista X2-X3: ELIMINADA


In [7]:
import numpy as np
import pandas as pd

np.random.seed(42)
N = 300

# A -> B, A -> C, B -> D, C -> D
A = np.random.randn(N)
B = 0.8 * A + 0.2 * np.random.randn(N)
C = -0.9 * A + 0.2 * np.random.randn(N)
D = 0.8 * B + 0.8 * C + 0.2 * np.random.randn(N)

df_4nodes = pd.DataFrame({'A': A, 'B': B, 'C': C, 'D': D})
print(df_4nodes.head())


          A         B         C         D
0  0.496714  0.231572 -0.295645  0.022477
1 -0.138264 -0.222648 -0.059995 -0.304782
2  0.647689  0.667610 -0.408999  0.212638
3  1.523030  1.340498 -1.099599  0.448409
4 -0.234153 -0.191503  0.293425  0.119757


In [12]:
from causallearn.search.ConstraintBased.FCI import fci

node_names = list(df_4nodes.columns)  # nombres de las variables
data_latent = df_4nodes.to_numpy()

graph, edges = fci(
        data_latent,
        independence_test_method='kci',
        alpha=0.05,
        depth=2,
        verbose=False,
        node_names=node_names,
        kernelX="Gaussian",
        kernelY="Gaussian",
        kernelZ="Gaussian",
        print_edges_to_trace=[('C','D')]
)

Depth=0, working on node 3: 100%|██████████| 4/4 [00:00<00:00, 206.79it/s]


Arista C-D: CIRCLE - CIRCLE
Arista C-D: CIRCLE - ARROW
Arista C-D: CIRCLE - CIRCLE
Arista C-D: CIRCLE - ARROW
Arista C-D: CIRCLE - ARROW
Arista C-D: CIRCLE - ARROW
Arista C-D: CIRCLE - ARROW
Arista C-D: CIRCLE - ARROW
Arista C-D: CIRCLE - ARROW
Arista C-D: CIRCLE - ARROW
Arista C-D: CIRCLE - ARROW
Arista C-D: CIRCLE - ARROW
Arista C-D: CIRCLE - ARROW


In [18]:
import numpy as np
import pandas as pd

np.random.seed(42)
N = 300

# Latent structure:
# L -> B, L -> C, A -> B, C -> D, D -> E
L = np.random.randn(N)
A = np.random.randn(N)

eps_B = np.random.randn(N)
eps_C = np.random.randn(N)
eps_D = np.random.randn(N)
eps_E = np.random.randn(N)

B = 0.9 * A + 0.9 * L + 0.2 * eps_B
C = 0.9 * L + 0.2 * eps_C
D = 0.9 * C + 0.2 * eps_D
E = 0.9 * D + 0.2 * eps_E

df_latent = pd.DataFrame({'A': A, 'B': B, 'C': C, 'D': D, 'E': E})
print(df_latent.head())


          A         B         C         D         E
0 -0.828995 -0.147655  0.520777  0.493745  0.600042
1 -0.560181 -0.813034 -0.203106 -0.268676 -0.352046
2  0.747294  1.429405  0.588669  0.554261  0.335195
3  0.610370  2.191188  1.626417  1.572435  1.414517
4 -0.020902 -0.146862 -0.172518 -0.145494 -0.164982


In [20]:
from causallearn.search.ConstraintBased.FCI import fci

node_names = list(df_latent.columns)  # nombres de las variables
data_latent = df_latent.to_numpy()

graph, edges = fci(
        data_latent,
        independence_test_method='kci',
        alpha=0.05,
        depth=2,
        verbose=False,
        node_names=node_names,
        kernelX="Gaussian",
        kernelY="Gaussian",
        kernelZ="Gaussian",
        print_edges_to_trace=[('B','C')]
)

Depth=0, working on node 4: 100%|██████████| 5/5 [00:00<00:00, 123.86it/s]


Arista B-C: CIRCLE - CIRCLE
Arista B-C: CIRCLE - ARROW
Arista B-C: CIRCLE - CIRCLE
Arista B-C: CIRCLE - ARROW
Arista B-C: CIRCLE - ARROW
Arista B-C: CIRCLE - ARROW
Arista B-C: CIRCLE - ARROW
Arista B-C: CIRCLE - ARROW
Arista B-C: CIRCLE - ARROW
Arista B-C: CIRCLE - ARROW
Arista B-C: CIRCLE - ARROW
Arista B-C: CIRCLE - ARROW
Arista B-C: CIRCLE - ARROW
